# Resume–Job Description Matching Agent
(A LangGraph LLM Agent)

Modules covered (matches LangGraph course till Module 4):


*   Memory & State
*   Multi-Agent Collaboration


*  Graph Execution
*  Human-style agent reasoning


*   LLM + Embeddings + Scoring + Looping

Overview
We are building a custom multi-analyst LangGraph agent that evaluates how well a resume matches a job description.

Instead of a single score, we simulate 3 evaluators, each with a different viewpoint:

* ATS Screener — keywords & format

* Hiring Manager — hands-on role fit
…* Runs 3 personas

* Produces a structured evaluation

In [1]:
%%capture --no-stderr
%pip install --quiet -U langgraph langchain_groq langchain_core langchain_community tavily-python


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.1.0 which is incompatible.
langchain 0.3.27 requires langchain-text-splitters<1.0.0,>=0.3.9, but you have langchain-text-splitters 1.0.0 which is incompatible.
langchain-google-genai 1.0.8 requires langchain-core<0.3,>=0.2.17, but you have langchain-core 1.1.0 which is incompatible.
langchain-openai 0.3.35 requires langchain-core<1.0.0,>=0.3.78, but you have langchain-core 1.1.0 which is incompatible.
langgraph-api 0.4.39 requires protobuf<7.0.0,>=6.32.1, but you have protobuf 4.25.8 which is incompatible.
replit 3.3.1 requires typing_extensions<4.0.0,>=3.7.4, but you have typing-extensions 4.12.2 which is incompatible.
replit 3.3.1 requires Werkzeug<3.0.0,>=2.0.0, but you have werkzeug 3.1.3 which is incompatible.
tf-keras 2.18

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "YOUR_KEY_HERE"
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"


In [4]:
from typing import TypedDict, List
from sentence_transformers import SentenceTransformer, util
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, END


Step 1 — Imports & Setup
We import embeddings, LLM, graph utilities, and state types.

In [ ]:
import os
os.environ["GROQ_API_KEY"] = "YOUR_KEY_HERE"

from typing import TypedDict, List, Dict, Any
from sentence_transformers import SentenceTransformer, util
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from concurrent.futures import ThreadPoolExecutor, as_completed
import textwrap
import copy

Step 2 — Embedding Model
This lets us compute numeric similarity.

In [6]:
# ---- Embedding similarity (you already had this) ----
embedder = SentenceTransformer("all-MiniLM-L6-v2")

Step 3 — Similarity Function
This converts both documents → vectors → cosine similarity.

In [7]:


def score_similarity(resume: str, job: str) -> float:
    vec1 = embedder.encode(resume, convert_to_tensor=True)
    vec2 = embedder.encode(job, convert_to_tensor=True)
    return float(util.pytorch_cos_sim(vec1, vec2)[0][0])





Step 4 — LLM Model
(We’re using Groq’s hosted LLaMA 3.1)